<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/meridian/blob/main/demo/Meridian_All_Features.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/Meridian_All_Features.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Meridian All Features Demo

This notebook demonstrates the end to end usage of the Meridian library. 
It loads sample data, configures a model, runs sampling, generates diagnostics, 
creates summary reports, performs budget optimization and shows how to save the model.

## Step 0: Install

In [ ]:
# Uncomment to install from PyPI
# !pip install --upgrade google-meridian[colab,and-cuda]

import tensorflow as tf
from psutil import virtual_memory
print(f'RAM: {virtual_memory().total/1e9:.1f} GB')
print('GPUs:', tf.config.list_physical_devices('GPU'))

## Step 1: Load the data

In [ ]:
from meridian.data import load
csv_url = 'https://raw.githubusercontent.com/google/meridian/main/meridian/data/simulated_data/csv/geo_all_channels.csv'
coord_to_columns = load.CoordToColumns(
    time='time',
    geo='geo',
    controls=['sentiment_score_control', 'competitor_sales_control'],
    population='population',
    kpi='conversions',
    revenue_per_kpi='revenue_per_conversion',
    media=[
        'Channel0_impression',
        'Channel1_impression',
        'Channel2_impression',
        'Channel3_impression',
        'Channel4_impression',
    ],
    media_spend=[
        'Channel0_spend',
        'Channel1_spend',
        'Channel2_spend',
        'Channel3_spend',
        'Channel4_spend',
    ],
    organic_media=['Organic_channel0_impression'],
    non_media_treatments=['Promo'],
)
correct_media_to_channel = {
    'Channel0_impression': 'Channel_0',
    'Channel1_impression': 'Channel_1',
    'Channel2_impression': 'Channel_2',
    'Channel3_impression': 'Channel_3',
    'Channel4_impression': 'Channel_4',
}
correct_media_spend_to_channel = {
    'Channel0_spend': 'Channel_0',
    'Channel1_spend': 'Channel_1',
    'Channel2_spend': 'Channel_2',
    'Channel3_spend': 'Channel_3',
    'Channel4_spend': 'Channel_4',
}
loader = load.CsvDataLoader(
    csv_path=csv_url,
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

## Step 2: Configure the model

In [ ]:
from meridian.model import model, spec, prior_distribution
import tensorflow_probability as tfp
from meridian import constants
roi_mu = 0.2
roi_sigma = 0.9
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)
mmm = model.Meridian(input_data=data, model_spec=model_spec)

## Step 3: Sample from the model

In [ ]:
mmm.sample_prior(500)
mmm.sample_posterior(n_chains=4, n_adapt=1000, n_burnin=500, n_keep=500, seed=1)

## Step 4: Diagnostics

In [ ]:
from meridian.analysis import visualizer
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

## Step 5: Summarize results

In [ ]:
from meridian.analysis import summarizer
mmm_summarizer = summarizer.Summarizer(mmm)
mmm_summarizer.output_model_results_summary('summary_output.html', '/content', '2021-01-25', '2024-01-15')

## Step 6: Run budget optimization

In [ ]:
from meridian.analysis import optimizer
budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize()
optimization_results.output_optimization_summary('optimization_output.html', '/content')

## Step 7: Save and load the model

In [ ]:
from meridian.model import load_mmm, save_mmm
file_path = '/content/saved_mmm.pkl'
model.save_mmm(mmm, file_path)
mmm = model.load_mmm(file_path)